## reading GoogleNet

## 1. 論文を読む際に注意すべき観点

- どんなもの？

ImageNet Large-Scale Visual Recognition Challenge 2014(ILSVRC14)にて優勝したモデル。<br>
Googleが作成したモデルであり、名前をGoogLeNetという。この分野の初期モデルLeNetをもじっている。

- 先行研究と比べてどこがすごい？

畳み込み層の重みをsparseにし，パラメータ数軽減させたinceptionというネットワーク内ネットワークを導入。<br>
精度を維持したまま、計算処理大幅に減らし、コンピュータリソースの効率化を実現させた。

- 技術や手法のキモはどこ？
複数の畳み込み層やpooling層から構成される Inceptionモジュール の導入。<br>
全結合層を利用せず、Global Average Pooling　の導入で、パラメータ数を大きく削減し，過学習を防いだ。

- どうやって有効だと検証した？

とにかく精度が出た

- 議論はある？

- 次に読むべき論文は？

TensorFlow のチュートリアル内では Inception-v3 モデルへの言及が多い.<br>
<br>
Rethinking the Inception Architecture for Computer Vision.<br>
https://arxiv.org/pdf/1512.00567.pdf

ResNet の論文<br>
K. He, X. Zhang, S. Ren, J. Sun.<br>
Deep Residual Learning for Image Recognition. arXiv:1512.03385, 2015.<br>
https://arxiv.org/pdf/1512.03385.pdf


## 2. 技術のキモ

### 2-1. Inceptionモジュール

GoogLeNetの一番の特徴は，複数の畳み込み層やpooling層から構成される **Inceptionモジュール** と呼ばれる小さなネットワーク (micro networks) を定義し，これを通常の畳み込み層のように重ねていくことで1つの大きなCNNを作り上げている点である．

本稿では，このような小さなネットワークをモジュールと呼ぶこととする．

この設計は，畳み込み層と多層パーセプトロン（実装上は1×1畳み込み）により構成されるモジュールを初めて利用した<br>
Network In Network (NIN)32に大きな影響を受けている．

<img src="15_imgnet/img201.png" width="450px" height="450px">

Inceptionモジュールでは，

- ネットワークを分岐させ，
- サイズの異なる畳み込みを行った後，
- それらの出力をつなぎ合わせる

という処理を行っている．<br>
この目的は，**畳み込み層の重みをsparseにし，パラメータ数のトレードオフを改善する** ことである．

そもそも畳み込み層は， sparseかつ共有された重みを持つ全結合層とみなす ことができるが，<br>
この畳み込み層自体も，「**入力チャネルとフィルタの重みとしてはdenseな結合をしている**」と言える．

これに対し，Inceptionモジュールは，この **入力チャネルとフィルタの重みがsparseになったもの** とみなすことができる．

すなわち，(b)のnaiveなInceptionモジュールは，<br>
maxpoolを除けば，本来は重みがsparseな5×5の畳み込み1つで表現することができる．

これを明示的にInceptionモジュールを利用することで， **遥かに少ないパラメータで同等の表現能力を持つ** CNNを構築することができる．<br>
このパラメータの削減は下図のように直感的に理解することができる．

すなわち，畳み込み層のパラメータ数はbias項を除くと<br>
  入力チャネル数×出力チャネル数×カーネルサイズ（e.g. 5x5=25）<br>
 で表現される．
 
 通常の畳み込みであれば，左のようにパラメータは全て何かしらの値を持っているためdenseである．
 
 一方，Inceptionでは異なるサイズの畳み込みを独立して行っているため，非0のパラメータ数が大きく減る．

<img src="15_imgnet/img202.png" width="450px" height="450px">

また，実際に利用されている(a)のInceptionモジュールでは，各畳み込み層の前に1×1の畳み込み層を挿入し，次元削減を行うことで更にパラメータを削減している．

### 2-2. Global Average Pooling

GoogLeNetではGlobal Average Pooling (GAP)が導入されている点にも注目したい．

従来のモデルは，「畳み込み層の後に複数の全結合層を重ねることで，最終的な1000クラス分類の出力を得る構造」<br>
となっていたが，これらの**全結合層はパラメータ数が多く，また過学習を起こす** ことが課題となっており，<br>
**dropoutを導入することで過学習を抑える必要** があった．

文献32で提案されているGAPは， 入力された特徴マップのサイズと同じサイズのaverage poolingを行うpooling層である<br>
（すなわち出力は1×1×チャネル数のテンソルとなる）．

文献32では，CNNの最後の畳み込み層の**出力チャネル数を最終的な出力の次元数（クラス数）と同一**とし，<br>
その後GAP（およびsoftmax）を適用することで，全結合層を利用することなく最終的な出力を得ることを提案している．

**全結合層を利用しないことで，パラメータ数を大きく削減し，過学習を防ぐ**ことができる．

下図にNINでのGAP処理を示す．

<img src="15_imgnet/img203.png" width="450px" height="450px">

GoogLeNetでは，最後の畳み込み層の出力チャネル数をクラス数と同一にすることはせず，<br>
GAPの後に全結合層を1層だけ適用し，最終出力を得る構成としている．

このGAPの利用は，現在ではクラス分類を行うCNNのベストプラクティスとなっている．

### 2-3. Auxiliary Loss

GoogLeNetの学習では，ネットワークの途中から分岐させた「サブネットワークにおいてもクラス分類」を行い，<br>
auxiliary loss(補助損失) を追加することが行われている．

これにより，ネットワークの中間層に直接誤差を伝搬させることで，**勾配消失を防ぐ**とともに<br>
**ネットワークの正則化** を実現している。

### 2-4. Inception-vX

文献33では，5×5の畳み込みを3×3の畳み込みを2つ重ねたものに置き換えることで更にInceptionモジュールの<br>
パラメータを削減したり，後述するbatch normalization28を導入したりする等の改良が行われたInception-v2,3が提案されている．

更に，文献34では，
- nn×1や1×nnの畳み込みを多数導入し，精度と計算量のトレードオフを改善したInception-v4や，
- ResNetの構造を取り入れたInception-ResNet

が提案されている．

## 3. 論文精読

### Abstract

We propose a deep convolutional neural network architecture codenamed Inception, which was responsible for setting the new state of the art for classification and detection in the ImageNet Large-Scale Visual Recognition Challenge 2014 (ILSVRC14). 

The main hallmark of this architecture is the improved utilization of the computing resources inside the network. 

Inception 深い畳み込みニューラルネットワークアーキテクチャ

特長は、ネットワーク内の**コンピューティングリソースの利用効率の向上**

This was achieved by a carefully crafted design that allows for increasing the depth and width of the network while keeping the computational budget constant. 

To optimize quality, the architectural decisions were based on the Hebbian principle and the intuition of multi-scale processing. 

One particular incarnation used in our submission for ILSVRC14 is called GoogLeNet, a 22 layers deep network, the quality of which is assessed in the context of classification and detection.

- 計算予算を一定に保ちながら
- ネットワークの深さと幅を増やすこと

を可能にする慎重に設計された設計

品質を最適化するために、アーキテクチャの決定は、Hebbianの原則とマルチスケール処理の直感に基づく

22層の深層ネットワークであるGoogLeNetと呼ばれた

### Hebbian Learning Rule

The Hebbian Learning Rule is a learning rule that specifies how much the weight of the connection between two units should be increased or decreased in proportion to the product of their activation. The rule builds on Hebbs's 1949 learning rule which states that the connections between two neurons might be strengthened if the neurons fire simultaneously. The Hebbian Rule works well as long as all the input patterns are orthogonal or uncorrelated. The requirement of orthogonality places serious limitations on the Hebbian Learning Rule. A more powerful learning rule is the delta rule, which utilizes the discrepancy between the desired and actual output of each output unit to change the weights feeding into it.

ヘッブの法則（ヘッブのほうそく）<br>
脳のシナプス可塑性についての法則である。ヘッブ則、ヘブ則とも呼ばれる。<br>
心理学者のドナルド・ヘッブによって提唱された。<br>
ニューロン間の接合部であるシナプスにおいて、シナプス前ニューロンの繰り返し発火によってシナプス後ニューロンに発火が起こると、そのシナプスの伝達効率が増強される。<br>
また逆に、発火が長期間起こらないと、そのシナプスの伝達効率は減退するというものである。

<img src="15_imgnet/img200.png" width="450px" height="450px">

### 1. Introduction

In the last three years, mainly due to the advances of deep learning, more concretely convolutional networks [10], the quality of image recognition and object detection has been progressing at a dramatic pace. 

One encouraging news is that most of this progress is not just the result of more powerful hardware, larger datasets and bigger models, but mainly a consequence of new ideas, algorithms and improved network architectures. 

過去3年間では、主に深い学習、より具体的には畳み込みネットワーク[10]の進歩により、画像認識と物体検出の品質は劇的な進歩を遂げている。

この進歩の大部分は、
- より強力なハードウェア、
- より大きいデータセット
- より大きいモデルの結果

だけでなく、主に
- 新しいアイデア、アルゴリズム、
- および改善されたネットワークアーキテクチャ

の結果である。

No new data sources were used, for example, by the top entries in the ILSVRC 2014 competition besides the classification dataset of the same competition for detection purposes. 

例えば、ILSVRC 2014競技のトップエントリーでは、検出目的で同じ競技会の分類データセットに加えて、<br>
新しいデータソースは使用されなかった。

Our GoogLeNet submission to ILSVRC 2014 actually uses 12× fewer parameters than the winning architecture of Krizhevsky et al [9] from two years ago, while being significantly more accurate. 

GoogLeNet は、AlexNet(2年前のKrizhevskyらのアーキテクチャ) より
- **12倍少ないパラメータを使用**
- 比較して、かなり正確

The biggest gains in object-detection have not come from the utilization of deep networks alone or bigger models, but from the synergy of deep architectures and classical computer vision, like the R-CNN algorithm by Girshick et al [6]. 

オブジェクト検出の最大の利点は、<br>
「ディープネットワーク単独またはより大きなモデルの利用からではなく」<br>
R-CNNアルゴリズム[6]のような深いアーキテクチャと古典的なコンピュータビジョンのシナジーからです。

Another notable factor is that with the ongoing traction of mobile and embedded computing, the efficiency of our algorithms – especially their power and memory use – gains importance. 

もう1つ注目すべき要因は、モバイルおよび組み込みコンピューティングの継続的な牽引により、アルゴリズムの効率、<br>
特に**電力とメモリの使用が重要**になります。

It is noteworthy that the considerations leading to the design of the deep architecture presented in this paper included this factor rather than having a sheer fixation on accuracy numbers. 

For most of the experiments, the models were designed to keep a computational budget of 1.5 billion multiply-adds at inference time, so that the they do not end up to be a purely academic curiosity, but could be put to real world use, even on large datasets, at a reasonable cost.

このホワイトペーパーで紹介したディープアーキテクチャーの設計につながる考慮事項には、精度の数値を固定するのではなく、この要素が含まれていました。

ほとんどの実験では、推論時に15億回の計算積算を行うように設計されていたため、純粋に学問的な好奇心に終わったわけではなく、実際の使用大規模なデータセットでは、妥当なコストで

In this paper, we will focus on an efficient deep neural network architecture for computer vision, codenamed Inception, which derives its name from the Network in network paper by Lin et al [12] in conjunction with the famous “we need to go deeper” internet meme [1].

In our case, the word “deep” is used in two different meanings: first of all, in the sense that we introduce a new level of organization in the form of the “Inception module” and also in the more direct sense of increased network depth.

効率的なディープニューラルネットワークアーキテクチャが主な論点。

論文では、「深い」という言葉は、2つの異なる意味で使用されています。<br>
1) 「インセプションモジュール」という新たな構造の中で<br>
2) 全体のネットワークにおいて<br>

In general, one can view the Inception model as a logical culmination of [12] while taking inspiration and guidance from the theoretical work by Arora et al [2].

The benefits of the architecture are experimentally verified on the ILSVRC 2014 classification and detection challenges, on which it significantly outperforms the current state of the art.

Aroraらの理論的研究[2]からヒントを得て、[12]の論理的な集大成として Inception モデルを見ることができる。

Inception アーキテクチャーの利点は、ILSVRC 2014の分類と検出の課題について実験的に検証されており、
現在の状態を大幅に上回っています。

### 2. Related Work

Starting with LeNet-5 [10], convolutional neural networks (CNN) have typically had a standard structure – stacked convolutional layers (optionally followed by contrast normalization and maxpooling) are followed by one or more fully-connected layers. 

Variants of this basic design are prevalent in the image classification literature and have yielded the best results to-date on MNIST, CIFAR and most notably on the ImageNet classification challenge [9, 21]. 

LeNet-5 [10]から、畳み込みニューラルネットワーク（CNN）は通常、標準的な構造を持っていました - 積み上げられた畳み込みレイヤー（オプションでコントラスト正規化とマックスプーリングが続いています）に1つ以上の完全接続レイヤーが続きます。

この基本的なデザインの変種は、画像分類文献でよく見られ、MNIST、CIFAR、そしてとりわけImageNet分類課題に関する最新の結果が得られている[9, 21]。

For larger datasets such as Imagenet, the recent trend has been to increase the number of layers [12] and layer size [21, 14], while using dropout [7] to address the problem of overfitting. 

Despite concerns that max-pooling layers result in loss of accurate spatial information, the same convolutional network architecture as [9] has also been successfully employed for localization [9, 14], object detection [6, 14, 18, 5] and human pose estimation [19]. 

最近の傾向<br>
Imagenetのような大きなデータセットの場合、オーバーフィットの問題に対処するために
- ドロップアウト[7]を使用しながら、
- レイヤー数[12] とレイヤーサイズ[21,14]を増やす

傾向があった。

マックスプーリング層が正確な空間情報を失うという懸念にもかかわらず、<br>
[9]と同じ畳み込みネットワークアーキテクチャでは

- 位置検出[9,14]
- 物体検出[6,14,18,5]
- 人間ポーズ推定[19]

で採用されている。

Inspired by a neuroscience model of the primate visual cortex, Serre et al. [15] use a series of fixed Gabor filters of different sizes in order to handle multiple scales, similarly to the Inception model. 

霊長類視覚野の神経科学モデルに触発されSerre et al [15]は、インセプションモデルと同様に、<br>
複数のスケールを扱うために、異なるサイズの固定**ガボールフィルタ**を使用します。

#### ガボールフィルタ

http://zellij.hatenablog.com/entry/20131003/p1

ガボール・フィルタは、画像処理で用いられる各種画像フィルタの1つ。

■ 画像フィルタ<br>
画像を構成する各画素の値に、特定の処理を施すことで、画像の処理を行うもの。

■ ガボール・フィルタってどんなもの？<br>
英語版WikipediaでGabor Filterの適用例として紹介されているのが下の図。<br>
http://en.wikipedia.org/wiki/Gabor_filter

ガボール・フィルタは<br>
「画像中にどの向きの線が含まれているかを抽出できるフィルタ」<br>
である。

However, contrary to the fixed 2-layer deep model of [15], all filters in the Inception model are learned. 

Furthermore, Inception layers are repeated many times, leading to a 22-layer deep model in the case of the GoogLeNet model.

しかし、[15]の固定された2層深いモデルとは異なり、 Inception　モデルのすべてのフィルタが学習されます。

さらに、Inceptionレイヤーは何度も繰り返され、GoogLeNetモデルの場合は22レイヤーの深いモデルになります。

Network-in-Network is an approach proposed by Lin et al. [12] in order to increase the representational
power of neural networks. 

When applied to convolutional layers, the method could be viewed as additional 1×1 convolutional layers followed typically by the rectified linear activation [9]. 

This enables it to be easily integrated in the current CNN pipelines. We use this approach heavily in our
architecture. 

Network-in-Networkは、Linら[12]が提案したアプローチです。 
ニューラルネットワークの表現力を高めるために。

畳み込み層に適用すると、 ReLU活性化関数に続く、追加の1×1畳み込み層と見なすことができる。

これにより、現在のCNNパイプラインに簡単に統合することができます。

However, in our setting, 1 × 1 convolutions have dual purpose: 
most critically, they are used mainly as dimension reduction modules to remove computational bottlenecks, that would otherwise limit the size of our networks. 

This allows for not just increasing the depth, but also the width of our networks without significant performance penalty.

1×1の畳み込みの二つの目的<br>
1) 次元削減 ; 計算のボトルネックを除去する<br>
2) ネットワークのサイズを制限する<br>
->  深さを増やすだけでなく、大幅なパフォーマンスの低下を招くことなくネットワークの幅を広げられる

The current leading approach for object detection is the Regions with Convolutional Neural Networks (R-CNN) proposed by Girshick et al. [6]. 

物体検出のための現在の主要なアプローチ<br>
はGirshickらによって提案された畳み込みニューラルネットワーク（R-CNN）を有する領域 [6]。

**Superpixel(スーパ画素)とは**<br>
輝度や色などの性質が似ている小領域.<br>
処理の単位要素をピクセルからSuperpixelに変更することで処理量を大幅に<br>
減少させることが可能なため,広く利用され始めている.

R-CNN decomposes the overall detection problem into two subproblems: <br>
to first utilize low-level cues such as color and superpixel consistency for potential object proposals in a category-agnostic fashion, 
and to then use CNN classifiers to identify object categories at those locations. 

R-CNNは、全体の検出問題を2つのサブ問題に分解する。<br>
1) 処理単位をスーパー画素に：<br>
　処理の単位要素をピクセルからSuperpixelに変更することで処理量を大幅に減少させる<br>
2) CNN分類器で物体のカテゴリを識別

**バウンディングボックス（Bounding Box）**とは<br>
図形をちょうど囲うのに必要な大きさの，四角い箱 （矩形）のこと。<br>
図形の大体の大きさを知るのに用いられます。

Such a two stage approach leverages the accuracy of bounding box segmentation with low-level cues, as well as the highly powerful classification power of state-of-the-art CNNs. 

We adopted a similar pipeline in our detection submissions, but have explored enhancements in both stages, such as multi-box [5] prediction for higher object bounding box recall, and ensemble approaches for better categorization of bounding box proposals.

2段階のアプローチは、<br>
1) 低レベルの手がかりによる矩形境界認識<br>
2) 最先端のCNNの非常に強力な分類力
を利用する。

我々は同様のパイプラインを採用したが、<br>
- マルチボックス[5]予測; より高いオブジェクト矩形再現のため
- 矩形分類のためのアンサンブルアプローチ

のような、両方の段階での強化を検討した。

### 3. Motivation and High Level Considerations

The most straightforward way of improving the performance of deep neural networks is by increasing their size.

This includes both increasing the depth – the number of levels – of the network and its width: the number of units at each level.

深いニューラルネットワークの性能を改善する最も簡単な方法は、そのサイズを大きくすることです。

これには、ネットワークの深さ（レベル数）と幅（各レベルのユニット数）の両方が含まれます。

This is as an easy and safe way of training higher quality models, especially given the availability of a large amount of labeled training data. 

これは、高品質のモデルを訓練するための簡単で安全な方法です。<br>
特に、大量のラベル付きトレーニングデータが利用可能であることが条件です。

However this simple solution comes with two major drawbacks.
Bigger size typically means a larger number of parameters, which makes the enlarged network more
prone to overfitting, especially if the number of labeled examples in the training set is limited.

This can become a major bottleneck, since the creation of high quality training sets can be tricky and expensive, especially if expert human raters are necessary to distinguish between fine-grained
visual categories like those in ImageNet (even in the 1000-class ILSVRC subset) as demonstrated
by Figure 1.

この単純な解決策(ネットワークのサイズを大きくする)には2つの大きな欠点<br>
1) **より多くのパラメータが必要**<br>
特に、トレーニングセットのラベル付きサンプルの数が限られている場合は、過学習する傾向<br>
-> 大きなボトルネックになる可能性

Another drawback of uniformly increased network size is the dramatically increased use of computational resources.

For example, in a deep vision network, if two convolutional layers are chained, any uniform increase in the number of their filters results in a quadratic increase of computation.

If the added capacity is used inefficiently (for example, if most weights end up to be close to zero),
then a lot of computation is wasted. 

Since in practice the computational budget is always finite, an efficient distribution of computing resources is preferred to an indiscriminate increase of size, even
when the main objective is to increase the quality of results.

2) **計算資源の使用が劇的に増加する**<br>
2つの畳み込み層が連鎖されている場合、フィルタ数が一様に増加すると、計算が2次的に増加する。

追加された容量が非効率的に使用される場合（たとえば、ほとんどの重みがゼロに近づく場合など）
多くの計算量が浪費される。

実際には計算予算は常に有限であるため、コンピューティングリソースの効率的な分配がサイズの無差別な増加よりも好ましい
主目的が結果の質を向上させる場合であっても。

The fundamental way of solving both issues would be by ultimately moving from fully connected
to sparsely connected architectures, even inside the convolutions. 

両方の問題を解決する基本的な方法<br>
全接続された構造からスパースに接続された構造に移行すること。

Besides mimicking biological systems, this would also have the advantage of firmer theoretical underpinnings due to the groundbreaking work of Arora et al. [2].

Their main result states that if the probability distribution of the data-set is representable by a large, very sparse deep neural network, then the optimal network topology can be constructed layer by layer by analyzing the correlation statistics of the activations of the last layer and clustering neurons with highly correlated outputs.

データセットの確率分布が大規模で非常に疎な深層ニューラルネットワークによって表現可能である場合、<br>
最後の層の活性化の相関統計を分析することによって最適なネットワークトポロジーを層ごとに構築することができ、<br>
相関の高い出力を有するニューロンをクラスタリングする。

Although the strict mathematical proof requires very strong conditions, the fact that this statement resonates with the well known Hebbian principle
 – neurons that fire together, wire together – suggests that the underlying idea is applicable even under less strict conditions, in practice.

厳密な数学的証明は非常に強い条件を必要とするが、ヘッブの法則と同じ<br>
「一緒に発火するほど、伝達効率はよくなる」

On the downside, todays computing infrastructures are very inefficient when it comes to numerical calculation on non-uniform sparse data structures. 

今日のコンピューティングインフラストラクチャは、<br>
不均一な疎なデータ構造の数値計算に関して非常に非効率的です。

Even if the number of arithmetic operations is reduced by 100×, the overhead of lookups and cache misses is so dominant that switching to sparse matrices would not pay off.

演算回数が100倍減っても、ルックアップやキャッシュミスのオーバーヘッドが非常に大きく、
疎な行列の計算に効果がない。

The gap is widened even further by the use of steadily improving, highly tuned, numerical libraries that allow for extremely fast dense matrix multiplication, exploiting the minute details of the underlying CPU or GPU hardware [16, 9].

Also, non-uniform sparse models require more sophisticated engineering and computing infrastructure.

不均一なスパースモデルでは、より洗練されたエンジニアリングおよびコンピューティング構造が必要です。

Most current vision oriented machine learning systems utilize sparsity in the spatial domain just by the virtue of employing convolutions.

However, convolutions are implemented as collections of dense connections to the patches in the earlier layer.

最新の画像認識の機械学習システムは、畳み込みの利用で、空間領域における希薄さを扱う。<br>
しかし、畳み込みは、前層のパッチへの密な接続のコレクションとして実装されます。

ConvNets have traditionally used random and sparse connection tables in the feature dimensions since [11] in order to break the symmetry and improve learning, the trend changed back to full connections with [9] in order to better optimize parallel computing.

The uniformity of the structure and a large number of filters and greater batch size allow for utilizing efficient dense computation.

ConvNetsは対称性を破壊し、学習を改善するために[11]以来、ランダムでスパースな接続テーブルを伝統的に使用

構造の均一性および多数のフィルタおよびより大きなバッチサイズは、効率的な密な計算を利用することを可能にする。

This raises the question whether there is any hope for a next, intermediate step: 
an architecture that makes use of the extra sparsity, even at filter level, as suggested by the theory, but exploits our current hardware by utilizing computations on dense matrices. 

The vast literature on sparse matrix computations (e.g. [3]) suggests that clustering sparse matrices into relatively dense submatrices tends to give state of the art practical performance for sparse matrix multiplication. 

It does not seem far-fetched to think that similar methods would be utilized for the automated construction of non-uniform deep-learning architectures in the near future.

次のステップへの希望があるのか<br>
フィルタレベルでさえ余分な希少性を利用するアーキテクチャですが、<br>
高密度行列計算を利用して現在のハードウェアを活用している。

スパース行列計算（例えば、[3]）に関する膨大な文献は、スパース行列を比較的高密度のサブ行列に<br>
クラスタリングすることは、スパース行列乗算の現実的な性能を最先端のものにする傾向があることを示唆している。

近い将来、同様の方法が非均一ディープ・ラーニング・アーキテクチャの自動構築に利用されるだろう。

The Inception architecture started out as a case study of the first author for assessing the hypothetical output of a sophisticated network topology construction algorithm that tries to approximate a sparse structure implied by [2] for vision networks and covering the hypothesized outcome by dense, readily available components.

インセプションアーキテクチャは、画像ネットワークを疎な構造に近似し、密なコンポーネントを容易に利用可能にする。

Despite being a highly speculative undertaking, only after two iterations on the exact choice of topology, we could already see modest gains against the reference architecture based on [12].

After further tuning of learning rate, hyperparameters and improved training methodology, we established that the resulting Inception architecture was especially useful in the context of localization and object detection as the base network for [6] and [5]. 

学習率、ハイパーパラメータ、トレーニング方法の調整を行った結果、Inceptionアーキテクチャは、[6] [5]の基盤ネットワークとしてのローカリゼーションとオブジェクト検出のコンテキストで特に有用であることを確認できた。

Interestingly, while most of the original architectural choices have been questioned and tested thoroughly, they turned out to be at least locally optimal.

少なくとも局所的には最適であることが判明しました。

One must be cautious though: although the proposed architecture has become a success for computer
vision, it is still questionable whether its quality can be attributed to the guiding principles that have lead to its construction.

アーキテクチャは画像認識で成功したが、品質がその構築につながった指針原則に帰することができるかどうか<br>
は依然として疑問である。

Making sure would require much more thorough analysis and verification:
for example, if automated tools based on the principles described below would find similar, but better topology for the vision networks.

徹底的な分析と検証が必要になる：<br>
例えば、以下に説明する原理に基づく自動化されたツールが、類似しているがより優れたトポロジーを見いだすことができるか。

The most convincing proof would be if an automated
system would create network topologies resulting in similar gains in other domains using the same
algorithm but with very differently looking global architecture.

At very least, the initial success of the Inception architecture yields firm motivation for exciting future work in this direction.

最も説得力のある証拠は、<br>
他のドメインで異なったグローバルアーキテクチャを備えたものが同様の成果をもたらすこと。

### 4. Architectural Details

The main idea of the Inception architecture is based on finding out how an optimal local sparse
structure in a convolutional vision network can be approximated and covered by readily available
dense components.

インセプションアーキテクチャの主な考え方<br>
畳み込みビジョンネットワークにおいて、高密度コンポーネントがいかにローカルなスパース構造で近似できるか

Note that assuming translation invariance means that our network will be built from convolutional building blocks.

All we need is to find the optimal local construction and to repeat it spatially.

ネットワークを不変変換すると仮定すると、複数の畳み込みのブロックから構築されることになります。

必要なのは、最適な局所構造を見つけ出し、それを空間的に繰り返すことだけ。

Arora et al. [2] suggests a layer-by layer construction in which one should analyze
the correlation statistics of the last layer and cluster them into groups of units with high correlation.

These clusters form the units of the next layer and are connected to the units in the previous layer. 

We assume that each unit from the earlier layer corresponds to some region of the input image and these
units are grouped into filter banks.

Arora ら[2]は以下を提案している。<br>
最後の層の相関統計を分析し、高い相関を有するユニットのグループにクラスタリングする。

これらのクラスタは、次の層のユニットを形成し、前層のユニットに接続される。

前のレイヤからの各ユニットは
- 入力画像のある領域に対応し、
- フィルタバンクにグループ化されている

と仮定します。

In the lower layers (the ones close to the input) correlated units would concentrate in local regions. 

下位層（入力に近い層）では、相関した単位は局所領域に集中する。

This means, we would end up with a lot of clusters concentrated in a single region and they can be covered by a layer of 1×1 convolutions in the next layer, as suggested in [12]. 

これは最終的に1つの領域に集中した多くのクラスタに分割されることになり、[12]で示唆されているように、<br>
次のレイヤで1×1の畳み込みのレイヤーでカバーできます。

However, one can also expect that there will be a smaller number of more spatially spread out clusters that can be covered by convolutions over larger patches, and there will be a decreasing number of patches over larger and larger regions. 

In order to avoid patchalignment issues, current incarnations of the Inception architecture are restricted to filter sizes 1×1, 3×3 and 5×5, however this decision was based more on convenience rather than necessity.

より大きなパッチの畳み込みによってクラスタ数を減らすことも期待できる。

パッチ配置問題を回避するために、インセプションアーキテクチャのフィルタサイズは1×1、3×3、5×5に<br>
制限されていますが、必要性よりも利便性に基づいています。

It also means that the suggested architecture is a combination of all those layers with their output filter banks concatenated into a single output vector forming the input of the next stage.

出力はレイヤのすべての組み合わせを単一ベクトルに連結したもの。

Additionally, since pooling operations have been essential for the success in current state of the art convolutional networks, it suggests that adding an alternative parallel pooling path in each such stage should have additional beneficial effect, too (see Figure 2(a)).

畳み込みネットワークでプーリング操作が不可欠であったため、各畳み込み層で平行プーリングを追加すると有益な結果が得られるはず。（図2（a）を参照）。

<img src="15_imgnet/img201.png" width="450px" height="450px">

As these “Inception modules” are stacked on top of each other, their output correlation statistics<br>
are bound to vary: <br>
as features of higher abstraction are captured by higher layers, their spatial concentration is expected to decrease suggesting that the ratio of 3×3 and 5×5 convolutions should increase as we move to higher layers.

これらの「インセプション・モジュール」が互いに積み重ねられているので、出力相関の統計値はさまざまに変化します。

抽象度の高い特徴ほど上位層で捕捉されるので、それらの空間濃度は減少すると予想され、<br>
より上位層にほど3×3および5×5の畳み込みの比が増加する。

One big problem with the above modules, at least in this naive form, is that even a modest number of 5×5 convolutions can be prohibitively expensive on top of a convolutional layer with a large number of filters. 

上記のモジュールの大きな問題の1つは、少なくともこの単純形式では、5×5の畳み込みでさえ、畳み込み層の上に<br>
多数のフィルタを使用すると非常に高価になる可能性があるということ。

This problem becomes even more pronounced once pooling units are added to the mix:<br>
their number of output filters equals to the number of filters in the previous stage.

この問題は、プールユニットがミックスに追加されると、さらに顕著になります。<br>
出力フィルタの数は、前層のフィルタの数に等しくなります。

The merging of the output of the pooling layer with the outputs of convolutional layers <br>
would lead to an inevitable increase in the number of outputs from stage to stage.

プーリング層の出力と畳み込み層の出力とのマージは、段階毎の出力の数が必然的に増加することにつながる。

Even while this architecture might cover the optimal sparse structure, it would do it very inefficiently,<br>
leading to a computational blow up within a few stages.

このアーキテクチャが最適な疎構造をカバーしているにもかかわらず、それは非常に非効率的であり、<br>
数段階で計算上の爆発を招く

This leads to the second idea of the proposed architecture: <br>
judiciously applying dimension reductions and projections wherever the computational requirements would increase too much otherwise. 

アーキテクチャの第2のアイデア<br>
計算量が膨大になる場合は、慎重に次元削減と投影を適用する。

This is based on the success of embeddings: even low dimensional embeddings might contain a lot of information about a relatively large image patch. 

However, embeddings represent information in a dense, compressed form and compressed information is harder to model.

低次元の埋め込みであっても、大きな画像パッチに関する多くの情報が含まれる。

しかし、埋め込みは情報を密な圧縮形式で表現し、圧縮情報はモデル化するのが難しい。

We would like to keep our representation sparse at most places (as required by the conditions of [2]) and compress the signals only whenever they have to be aggregated en masse.

That is, 1×1 convolutions are used to compute reductions before the expensive 3×3 and 5×5 convolutions. 

Besides being used as reductions, they also include the use of rectified linear activation which makes them dual-purpose. The final result is depicted in Figure 2(b).

ほとんどの場所で（[2]の条件で必要とされるように）我々の表現を疎にして保ち、一括集約の必要があるときにのみ圧縮したい。

すなわち、高価な3×3および5×5の畳み込みの前に、1×1の畳み込みを使用して縮小を計算する。

削減と同時に、ReLU活性化関数を使用。

In general, an Inception network is a network consisting of modules of the above type stacked upon each other,
with occasional max-pooling layers with stride 2 to halve the resolution of the grid.

一般的に、インセプションネットワークは、上記のタイプのモジュールを互いに積み重ねたネットワークであり、<br>
グリッドの分解能を半分にするためにストライド2で時折max-poolingレイヤーがあります。

For technical reasons (memory efficiency during training), it seemed beneficial to start using Inception
modules only at higher layers while keeping the lower layers in traditional convolutional fashion.

This is not strictly necessary, simply reflecting some infrastructural inefficiencies in our current implementation.

技術的な理由（トレーニング中のメモリ効率）から、<br>
Inceptionモジュールは、上位層でのみ使用し、下位層は従来の畳み込み方式で維持することが有益。

これは厳密には必要ではなく、現行の実装におけるインフラストラクチャの非効率性を反映するだけ。

One of the main beneficial aspects of this architecture is that it allows for increasing the number of units at each stage significantly without an uncontrolled blow-up in computational complexity. 

The ubiquitous use of dimension reduction allows for shielding the large number of input filters of the last stage to the next layer, first reducing their dimension before convolving over them with a large patch size. 

Another practically useful aspect of this design is that it aligns with the intuition that visual information should be processed at various scales and then aggregated so that the next stage can abstract features from different scales simultaneously.

アーキテクチャのメリット<br>
計算上の複雑さを増やすことなく、各ステージでのユニット数を大幅に増やすことができる

最終段で多数の入力フィルタを使用せずに、次元削減してから大きなパッチ・サイズで畳み込む。

実用的有用な側面は、視覚情報を様々なスケールで処理し、次のステージが異なるスケールのフィーチャを同時に抽象化できるように集約するという直観と一致すること。

The improved use of computational resources allows for increasing both the width of each stage
as well as the number of stages without getting into computational difficulties.

計算リソースの改善で、各ステージの幅を増やせる。

Another way to utilize the inception architecture is to create slightly inferior, but computationally cheaper versions of it.

インセプションアーキテクチャを利用するもう1つの方法は、<br>
やや劣っているが計算的に安価なバージョンを作成すること。

We have found that all the included the knobs and levers allow for a controlled balancing of computational resources that can result in networks that are 2 − 3× faster than similarly performing
networks with non-Inception architecture, however this requires careful manual design at this point.

計算資源のバランシングを可能にし、インセプションなしの構造に比べて2〜3倍高速。

しかし、注意深い手動設計が必要。

### 5. GoogLeNet

We chose GoogLeNet as our team-name in the ILSVRC14 competition.<br>
This name is an homage to Yann LeCuns pioneering LeNet 5 network [10]. <br>
We also use GoogLeNet to refer to the particular incarnation of the Inception architecture used in our submission for the competition. 

We have also used a deeper and wider Inception network, the quality of which was slightly inferior, 
but adding it to the ensemble seemed to improve the results marginally. 

より深くより広いInceptionネットワークを使用。その品質はやや劣っていましたが、アンサンブルで結果をわずかに改善した。

We omit the details of that network, since our experiments have shown that the influence of the exact architectural parameters is relatively minor. 

実験の結果、正確なアーキテクチャパラメータの影響は比較的小さいため、ネットワークの詳細は省略している。

Here, the most successful particular instance (named GoogLeNet) is described in Table 1 for demonstrational purposes.

The exact same topology (trained with different sampling methods) was used for 6 out of the 7 models in our ensemble.

最も成功した特定のインスタンス（GoogLeNetという名前）をデモンストレーション目的で表1に示す。

アンサンブルで、7つのモデルのうち6つが完全に同じトポロジ（異なるサンプリング方法で訓練された）が使用された。

All the convolutions, including those inside the Inception modules, use rectified linear activation.

Inceptionモジュール内部を含むすべての畳み込みは、ReLU活性化関数を使用。

The size of the receptive field in our network is 224×224 taking RGB color channels with mean subtraction.

“#3×3 reduce” and “#5×5 reduce” stands for the number of 1×1 filters in the reduction layer used before the 3×3 and 5×5 convolutions. 

One can see the number of 1×1 filters in the projection layer after the built-in max-pooling in the pool proj column. 
All these reduction/projection layers use rectified linear activation as well.

我々のネットワークにおける受容野の大きさは、平均減算を伴うRGBカラーチャンネルを取る224×224である。

"＃3×3リダクション"と "＃5×5リダクション"とは、3×3および5×5畳み込みの前に使用される削減層の1×1フィルタの数を表します。

pool proj列に組み込まれているマックスプーリング後の投影レイヤーの1×1フィルターの数を見ることができます。
これらの削減/投影層はすべて、ReLU活性化関数使用。

The network was designed with computational efficiency and practicality in mind, so that inference can be run on individual devices including even those with limited computational resources, especially with low-memory footprint. 

ネットワークは計算効率と実用性を念頭に置いて設計されているため、限られた計算リソースを持つもの、特にメモリ不足のフットプリントを含む個々のデバイスでも推論を実行できます。

The network is 22 layers deep when counting only layers with parameters (or 27 layers if we also count pooling).

パラメータを持つ層は22層、プーリング含めて27層

The overall number of layers (independent building blocks) used for the construction of the network is about 100.

独立ブロック含め、ネットワークの構築に使用される層の総数は約100です。

However this number depends on the machine learning infrastructure system used.

層数は、使用される機械学習インフラに依存する。

The use of average pooling before the classifier is based on [12], although our implementation differs in that we use an extra linear layer.

分類器の前に平均プーリングを使用[12]し、線形レイヤ(softmaxで分類)を付与している点で異なる。

This enables adapting and fine-tuning our networks for other label sets easily, but it is mostly convenience and we do not expect it to have a major effect. 

It was found that a move from fully connected layers to average pooling improved the top-1 accuracy by about 0.6%, however the use of dropout remained essential even after removing the fully connected layers.

全結合を利用せずに平均プーリング(Global Average Pooling)を行うと、トップ-1の精度は約0.6％向上した。<br>
しかし、全結合を削除してもドロップアウトの使用は重要だった。

Given the relatively large depth of the network, the ability to propagate gradients back through all the layers in an effective manner was a concern. 

ネットワークの深さが比較的大きいことを考えると、効果的な方法ですべての層に勾配逆伝播できるか懸念があった。

One interesting insight is that the strong performance of relatively shallower networks on this task suggests that the features produced by the layers in the middle of the network should be very discriminative.

比較的浅いネットワークなら、中間層で生成される特徴量が非常に差別的である。

By adding auxiliary classifiers connected to these intermediate layers, we would expect to encourage discrimination in the lower stages in the classifier, increase the gradient signal that gets propagated back, and provide additional regularization.

中間層に補助分類器を追加することによって、分類器の下位で伝播する勾配信号を増加させ、追加の正則化が可能に。

These classifiers take the form of smaller convolutional networks put on top of the output of the Inception (4a) and (4d) modules.

この補助分類器は、インセプション（4a）および（4d）モジュールの出力の上位に位置し、より小さな畳み込みネットワークの形を取る。

During training, their loss gets added to the total loss of the network with a discount weight (the losses of the auxiliary classifiers were weighted by 0.3). 

At inference time, these auxiliary networks are discarded.

訓練中に、補助分類器の損失は重みづけられネットワークの全損失に追加される。<br>
（補助分類器の損失は0.3で重み付けした）。

推論時には、これらの補助ネットワークは破棄されます。

The exact structure of the extra network on the side, including the auxiliary classifier, is as follows:

- An average pooling layer with 5×5 filter size and stride 3, resulting in an 4×4×512 output for the (4a), and 4×4×528 for the (4d) stage.
- A 1×1 convolution with 128 filters for dimension reduction and rectified linear activation.
- A fully connected layer with 1024 units and rectified linear activation.
- A dropout layer with 70% ratio of dropped outputs.
- A linear layer with softmax loss as the classifier (predicting the same 1000 classes as the main classifier, but removed at inference time).

A schematic view of the resulting network is depicted in Figure 3.


### 6. Training Methodology

Our networks were trained using the DistBelief [4] distributed machine learning system using modest amount of model and data-parallelism. 

Although we used CPU based implementation only, a rough estimate suggests that the GoogLeNet network could be trained to convergence using few high-end GPUs within a week, the main limitation being the memory usage. 

私たちのネットワークは、控えめな量のモデルとデータ並列性を使用したDistBelief [4]分散マシン学習システムを使用して訓練されました。

CPUベースの実装のみを使用していましたが、大まかな見積もりによれば、GoogLeNetネットワークは1週間以内に数個のハイエンドGPUを使用して収束するように訓練することができます。

Our training used asynchronous stochastic gradient descent with 0.9 momentum [17], fixed learning rate schedule (decreasing the learning rate by 4% every 8 epochs). 

Polyak averaging [13] was used to create the final model used at inference time.

訓練では、非同期確率勾配降下を用いて0.9モーメンタム[17]、学習率固定（学習率を8エポックごとに4％減少させた）。

推論時に使用される最終モデルの作成には、Polyak averaging [13]を使用しました。

Our image sampling methods have changed substantially over the months leading to the competition, 
and already converged models were trained on with other options, sometimes in conjunction with changed hyperparameters, like dropout and learning rate, so it is hard to give a definitive guidance to the most effective single way to train these networks. 

私たちの画像サンプリング方法は、競争に至るまでの数ヶ月にわたって大幅に変化しました。

ドロップアウトや学習率などハイパーパラメータを変更し、訓練に最も効果的な指針を与えることは難しい。

To complicate matters further, some of the models were mainly trained on smaller relative crops, others on larger ones, inspired by [8].

いくつかのモデルは相対的に小さなデータで訓練され、より大きなデータで訓練した。

Still, one prescription that was verified to work very well after the competition includes sampling of various sized patches of the image whose size is distributed evenly between 8% and 100% of the image area and whose aspect ratio is chosen randomly between 3/4 and 4/3. 

競技会後に非常にうまく動作することが確認された対応の一つ<br>
サイズが8％から100％の間で均等分布している様々なサイズのパッチを、<br>
アスペクト比が3/4〜4/3の間でランダムにサンプリング選択すること。

Also, we found that the **photometric distortions** by Andrew Howard [8] were useful to combat overfitting to some extent.

In addition, we started to use random interpolation methods (bilinear, area, nearest neighbor and cubic, with equal probability) for resizing relatively late
 and in conjunction with other hyperparameter changes, so we could not tell definitely whether the final results were affected positively by their use.

Andrew Howard [8]の測光歪みは過学習対策にある程度有用であった。

さらに、比較的遅くサイズを変更するために、ランダム補間法（双線形、面積、最近傍と立方体、等しい確率で）を
使用して他のハイパーパラメータの変更と併せても、プラスの効果は見出せなかった。

### 9. Conclusions

Our results seem to yield a solid evidence that approximating the expected optimal sparse structure by readily available dense building blocks is a viable method for improving neural networks for computer vision.

容易に利用可能な高密度なビルディングブロックを、最適な疎構造で近似することで、
画像認識のニューラルネットワークを改善する有益な方法である。

The main advantage of this method is a significant quality gain at a modest increase of computational requirements compared to shallower and less wide networks. 

この方法の主な利点<br>
浅くて幅の狭いネットワークと比較して、わずかな計算増加ではかなりの品質向上している。

Also note that our detection work was competitive despite of neither utilizing context nor performing bounding box regression and this fact provides further evidence of the strength of the Inception architecture. 

また、コンテキストの利用やバウンディングボックスの回帰を行っていないにもかかわらず、我々の検出作業は競争力があり、この事実はInceptionアーキテクチャの強さの証拠となります。

Although it is expected that similar quality of result can be achieved by much more expensive networks of similar depth and width, our approach yields solid evidence that moving to sparser architectures is feasible and useful idea in general.

This suggest promising future work towards creating sparser and more refined structures in automated ways on the basis of [2].

同様の深さと幅を持つはるかに高価なネットワークでも同様の品質の結果が得られると期待されていますが、<br>
より疎のアーキテクチャへの移行は一般的に実現可能で有用なアイデアです。


## 4. 論文精読(Rethinking the Inception Architecture for Computer Vision)

ネットワーク自体はgooglenetを順当に拡張していったもの。<br>
この論文ではネットワークの設計方針を言語化している。

- downsamplingする前には、ネットワークのチャンネル数を増やす

VGGNet [18]はアーキテクチャ上の単純さの魅力的な特長を持っていますが、これには高いコストがかかります。ネットワークを評価するには多くの計算が必要です。

一方、GoogLeNet [20]のインセプションアーキテクチャは、メモリと計算予算の厳格な制約の下でもうまく機能するように設計されています。

たとえば、GoogleNetは500万のパラメータしか使用していませんでしたが、これは前のバージョンのAlexNetに比べて12倍の削減で、6,000万のパラメータを使用しました。

さらに、VGGNetはAlexNetよりも約3倍のパラメータを使用していました。
インセプションの計算コストは​​、VGGNetやその高性能サクセサ[6]よりもはるかに低い。

これにより、巨額のデータシナリオ[17]、[13]では、巨額のデータを妥当なコストで処理する必要があったインシナレーションネットワークを利用することが可能になりました。
計算能力は、例えば、移動ビジョン設定において、本質的に制限されます。

ターゲットメモリの使用[2]、[15]に特化したソリューションを適用したり、計算トリック[10]を介して特定の演算の実行を最適化することによって、これらの問題の一部を緩和することは確かに可能です。

しかしながら、これらの方法は複雑さが増大します。<br>
さらに、これらの方法は、インセプションアーキテクチャを最適化するためにも適用でき、<br>
効率ギャップを再び広げることができます。

しかし、インセプションアーキテクチャの複雑さは、ネットワークを変更することをより困難にしています。

アーキテクチャーが純粋にスケールアップされていれば、計算上の利益の大部分はすぐに失われる可能性があります。また、[20]では、GoogLeNetアーキテクチャのさまざまな設計上の決定につながる寄与要因についての明確な説明はありません。

これにより、効率を維持しながら新しいユースケースに適合させることが非常に難しくなります。例えば、
いくつかのインセプションスタイルのモデルの容量を増やす必要があると思われる場合、すべてのフィルタバンクサイズの数を倍にするだけの簡単な変換は、計算コストとパラメータ数の両方を4倍に増加させます。

これは、多くの実用的なシナリオにおいて、特に関連する利益が控えめである場合には、非常に不合理であることが判明する可能性がある。

このホワイトペーパーでは、いくつかの一般的な原則と、それが証明した最適化のアイデアについて説明します
効率的な方法で畳み込みネットワークを拡大するのに便利です。

私たちの原則はInceptiontypeネットワークに限定されるものではありませんが、Inceptionスタイルのビルディングブロックの一般的な構造は、自然にそれらの制約を組み込むのに十分柔軟であるため、そのコンテキストではより簡単に観察できます。

これは、インセプションモジュールの寸法縮小および平行構造の広範な使用によって可能になり、近くのコンポーネントに対する構造変化の影響を緩和することができます。

それでも、モデルの高品質を維持するためにはいくつかの指針が守られなければならないので、そうすることについては慎重である必要があります。

### 4-2. General Design Principles

畳み込みネットワークを使用したアーキテクチャ上の選択を伴う大規模な実験に基づくいくつかの設計原則について説明します。

この時点で、以下の原則の有用性は推測的であり、有効性の正確さと有効性の範囲を評価するためには、今後の実験的証拠が追加で必要となる。

依然として、これらの原則からの重大な逸脱は、ネットワークの品質の低下を招き、その逸脱が検出された状況を修正する結果、一般的にアーキテクチャが改善されていました。

#### 1. Avoid representational bottlenecks, especially early in the network. 

Feed-forward networks can be represented by an acyclic graph from the input layer(s) to the classifier or regressor. 

This defines a clear direction for the information flow. For any cut separating the inputs from the outputs, one can access the amount of information passing though the cut. 

One should avoid bottlenecks with extreme compression. 

In general the representation size should gently decrease from the inputs to the outputs before reaching the final representation used for the task at hand. 

Theoretically, information content can not be assessed merely by the dimensionality of the representation as it discards important factors like correlation structure; the dimensionality merely provides a rough estimate of information content.


フィードフォワードネットワークは、入力層から分類器/回帰器への非循環グラフによって表すことができる。

これは、情報フローの明確な方向を定義します。入力を出力から分離する任意の切断については、<br>
切断を通って通過する情報の量にアクセスすることができる。

極端な圧縮のボトルネックを避ける必要があります。

**ダウンサンプリングする前に、入力から出力まで緩やかに減少する＝チャネル数を増やす**必要があります。

理論的には、情報の内容は、相関構造のような重要な要素を捨てるだけなので、表現の次元によって評価することはできません。次元は単に情報内容の概算を提供するだけである。

#### 2. Higher dimensional representations are easier to process locally within a network.

Increasing the activations per tile in a convolutional network allows for more disentangled features. 

The resulting networks will train faster.

高い次元表現は、ネットワーク内で局所的に処理する方が簡単です。

畳み込みネットワークにおいてタイル当たりの活性化を増加させることにより、より絡み合った特徴が可能になる。

結果として生じるネットワークはより速く訓練できる。

#### 3. Spatial aggregation can be done over lower dimensional embeddings without much or any loss in representational power. 

For example, before performing a more spread out (e.g. 3 × 3) convolution, one can reduce the dimension of the input representation before the spatial aggregation without expecting serious adverse effects. 

We hypothesize that the reason for that is the strong correlation between adjacent unit results in much less loss of information during dimension reduction, if the outputs are used in a spatial aggregation context. 

Given that these signals should be easily compressible, the dimension reduction even promotes faster learning.



空間的集約は、表現力の低下をほとんどまたはまったく伴わずに、より低次元の埋め込みに対して行うことができる。

例えば、より広がりのある（例えば3×3）畳み込みを実行する前に、重大な悪影響を期待することなく空間集約の前に入力表現の次元を縮小することができる。

その理由は、出力が空間集約コンテキストで使用されている場合、隣接ユニット間の強い相関関係が、次元削減中の情報損失を大幅に減少させるという仮説です。

これらの信号が容易に圧縮可能でなければならないことを考えると、次元の縮小は学習の迅速化を促進する。

#### 4. Balance the width and depth of the network. 

Optimal performance of the network can be reached by balancing the number of filters per stage and the depth of the network. 

Increasing both the width and the depth of the network can contribute to higher quality networks.<br>
However, the optimal improvement for a constant amount of computation can be reached if both are increased in parallel. 

The computational budget should therefore be distributed in a balanced way between the depth and width of the network.


4.ネットワークの幅と深さのバランス

ネットワークの最適なパフォーマンスは、ステージごとのフィルタ数とネットワークの深さのバランスを取ることで達成できます。

ネットワークの幅と深さを増やすと、より高品質になる。<br>
しかし、両方を並行して増加させると、計算量の利用上限に達してしまう。

したがって、計算予算は、ネットワークの深さと幅との間にバランスのとれた方法で分散されるべきである。

### 4-3. Factorizing Convolutions with Large Filter Size

Much of the original gains of the GoogLeNet network
[20] arise from a very generous use of dimension reduction.

This can be viewed as a special case of factorizing
convolutions in a computationally efficient manner. Consider for example the case of a 1 × 1 convolutional layer followed by a 3 × 3 convolutional layer. 

In a vision network, it is expected that the outputs of near-by activations are highly correlated. 
Therefore, we can expect that their activations can be reduced before aggregation and that this should result in similarly expressive local representations.

Here we explore other ways of factorizing convolutions in various settings, especially in order to increase the computational efficiency of the solution. 

Since Inception networks are fully convolutional, each weight corresponds to one multiplication per activation. 

Therefore, any reduction in computational cost results in reduced number of parameters.

This means that with suitable factorization, we can
end up with more disentangled parameters and therefore with faster training. 

Also, we can use the computational and memory savings to increase the filter-bank sizes of our network while maintaining our ability to train each model replica on a single computer.

3.ラージフィルタによる畳み込みコンボリューションの因数分解サイズ

GoogLeNetネットワークのオリジナルの利点の大部分[20]は次元削減の非常に寛大な使用から生じる。

これは因数分解の特殊なケースと見ることができます
畳み込みを計算的に効率的に行うことができる。例えば、1×1畳み込み層の後に3×3畳み込み層が続く場合を考える。

ビジョンネットワークでは、近くにあるアクティベーションの出力は高度に相関すると予想されます。
したがって、集約前にアクティベーションを減らすことができ、同様に表現的な表現をもたらすはずです。

ここでは、特にソリューションの計算効率を向上させるために、さまざまな設定で畳み込みを因数分解する他の方法を検討します。

インセプションネットワークは完全に畳み込み型であるため、各重みは1回の起動につき1回の乗算に対応します。

したがって、計算コストの低減は、パラメータの数の減少をもたらす。

これは、適切な因数分解を用いると、
より解き放たれたパラメータで、したがってより速いトレーニングで終わります。

また、計算機とメモリの節約を使用して、ネットワークのフィルタバンクサイズを拡大しながら、各モデルレプリカを1台のコンピュータでトレーニングできるようにします。

## 参考文献

Going deeper with convolutions. arXiv: 1409.4842, 2014. <br>
https://arxiv.org/pdf/1409.4842.pdf

Rethinking the Inception Architecture for Computer Vision. arXiv: 1512.00567, 2015.<br>
https://arxiv.org/pdf/1512.00567.pdf

Hebbian Learning Rule<br>
http://penta.ufrgs.br/edu/telelab/3/hebbian_.htm

ヘッブの法則<br>
https://goo.gl/94UFzk

畳み込みニューラルネットワークの最新研究動向 (〜2017)<br>
https://qiita.com/yu4u/items/7e93c454c9410c4b5427


GoogLeNetのChainerモデル<br>
https://github.com/leetenki/googlenet_chainer

ディープラーニングの基礎技術と今後の課題・展望　～画像認識分野を中心に～<br>
https://www.slideshare.net/nlab_utokyo/20150930-53741757

ガボールフィルタ<br>
http://zellij.hatenablog.com/entry/20131003/p1

Gabor filter<br>
https://en.wikipedia.org/wiki/Gabor_filter